In [240]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import os
import numpy as np
import math
import re

Steam works API data

In [241]:
data = pq.read_table('../data/steam_app_details.parquet')
steamworks_df = pa.Table.to_pandas(data)
steamworks_df.head()

,appid,name,developers,publishers,initial_price,final_price,platforms,metacritic,genres,release_date
0,10,Counter-Strike,[Valve],[Valve],999.0,999.0,"{'linux': True, 'mac': True, 'windows': True}",88.0,"[{'description': 'Action', 'id': '1'}]","Nov 1, 2000"
1,20,Team Fortress Classic,[Valve],[Valve],499.0,499.0,"{'linux': True, 'mac': True, 'windows': True}",NaN,"[{'description': 'Action', 'id': '1'}]","Apr 1, 1999"
2,30,Day of Defeat,[Valve],[Valve],499.0,499.0,"{'linux': True, 'mac': True, 'windows': True}",79.0,"[{'description': 'Action', 'id': '1'}]","May 1, 2003"
3,40,Deathmatch Classic,[Valve],[Valve],499.0,499.0,"{'linux': True, 'mac': True, 'windows': True}",NaN,"[{'description': 'Action', 'id': '1'}]","Jun 1, 2001"
4,50,Half-Life: Opposing Force,[Gearbox Software],[Valve],499.0,499.0,"{'linux': True, 'mac': True, 'windows': True}",NaN,"[{'description': 'Action', 'id': '1'}]","Nov 1, 1999"


In [242]:
steamworks_df['appid'] = steamworks_df['appid'].astype(int)

In [243]:
steamworks_df.dtypes

appid              int64
name              object
developers        object
publishers        object
initial_price    float64
final_price      float64
platforms         object
metacritic       float64
genres            object
release_date      object
dtype: object

In [244]:
len(steamworks_df)

18096

Cleaning problematic names/developers/publishers/genres

In [245]:
print(type(steamworks_df['developers'].iloc[0]))
print(type(steamworks_df['publishers'].iloc[0]))
print(type(steamworks_df['genres'].iloc[0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [246]:
steamworks_df['developers'] = steamworks_df['developers'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
steamworks_df['publishers'] = steamworks_df['publishers'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
steamworks_df['genres'] = steamworks_df['genres'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)


In [247]:
steamworks_df['name'] = steamworks_df['name'].str.title()
steamworks_df['publishers'] = steamworks_df['publishers'].apply(lambda pubs: [pub.title() for pub in pubs] if isinstance(pubs, list) else pubs)
steamworks_df['developers'] = steamworks_df['developers'].apply(lambda devs: [dev.title() for dev in devs] if isinstance(devs, list) else devs)

In [248]:
def is_ascii(s):
    return bool(re.match(r'^[\x00-\x7F]*$', s))

In [249]:
steamworks_df = steamworks_df[steamworks_df['name'].apply(is_ascii)]
steamworks_df = steamworks_df[steamworks_df['developers'].apply(lambda devs: devs != [] and all(is_ascii(dev) for dev in devs))]
steamworks_df = steamworks_df[steamworks_df['publishers'].apply(lambda pubs: pubs != [] and all(is_ascii(pub) for pub in pubs))]


In [250]:
steamworks_df = steamworks_df[~steamworks_df['publishers'].apply(lambda pubs: 'N/A' in pubs)]

normalizing genres

In [251]:
print(set([genre['description'] for genres in steamworks_df['genres'] for genre in genres]))

{'Game Development', 'Design & Illustration', 'Utilities', 'Indie', 'Animation & Modeling', 'Accesso anticipato', 'Бойовики', 'Violent', 'Strategy', 'Racing', 'Симулятори', 'Simulazione', 'Simulation', 'Casual', 'Adventure', 'Sports', 'RPG', 'Рольові ігри', 'Action', 'Early Access', 'Пригоди', 'Massively Multiplayer', 'Free To Play', 'Education'}


In [252]:
genre_translation = {
    'Бойовики': 'Action',
    'Пригоди': 'Adventure',
    'Рольові ігри': 'RPG',
    'Симулятори': 'Simulation',
    'Simulazione': 'Simulation',
    'Accesso anticipato': 'Early Access'
}

for index, row in steamworks_df.iterrows():
    for genre in row['genres']:
        if genre['description'] in genre_translation:
            genre['description'] = genre_translation[genre['description']]
print(set([genre['description'] for genres in steamworks_df['genres'] for genre in genres]))

{'Game Development', 'Design & Illustration', 'Massively Multiplayer', 'Strategy', 'Free To Play', 'Racing', 'Adventure', 'Violent', 'Education', 'Utilities', 'Sports', 'Indie', 'RPG', 'Animation & Modeling', 'Simulation', 'Action', 'Early Access', 'Casual'}


In [253]:
target_genres = ['Animation & Modeling', 'Design & Illustration', 'Utilities', 'Game Development', 'Violent', 'Education']
matching_rows = steamworks_df[
    steamworks_df['genres'].apply(lambda genre_list: any(genre['description'] in target_genres for genre in genre_list))
]

In [254]:
matching_rows

,appid,name,developers,publishers,initial_price,final_price,platforms,metacritic,genres,release_date
1153,711570,Epic Battle Simulator 2,[Rappid Studios],[Rappid Studios],499.0,499.0,"{'linux': False, 'mac': False, 'windows': True}",NaN,"[{'description': 'Violent', 'id': '73'}, {'des...","Nov 2, 2017"
7399,2308700,Gb Basic,[Tony Wang],[Tony Wang],NaN,NaN,"{'linux': True, 'mac': False, 'windows': True}",NaN,"[{'description': 'Indie', 'id': '23'}, {'descr...",To be announced
9750,2645040,Cargo,"[Scythe, Killswitch Games]",[Killswitch Entertainment],NaN,NaN,"{'linux': False, 'mac': False, 'windows': True}",NaN,"[{'description': 'Indie', 'id': '23'}, {'descr...",Coming soon
10376,2721170,Brutal Ball,"[Lucid Matrix Studios, Inc.]","[Lucid Matrix Studios, Inc.]",NaN,NaN,"{'linux': False, 'mac': False, 'windows': True}",NaN,"[{'description': 'Action', 'id': '1'}, {'descr...","Apr 4, 2024"
16797,3267430,Last Remains,"[Earn Alliance, Renderer Game Studios]",[Earn Alliance],NaN,NaN,"{'linux': False, 'mac': False, 'windows': True}",NaN,"[{'description': 'Action', 'id': '1'}, {'descr...",January 2025


In [255]:
steamworks_df['genres'] = steamworks_df['genres'].apply(
        lambda genre_list: [genre for genre in genre_list if genre['description'] not in target_genres])

In [256]:
print(set([genre['description'] for genres in steamworks_df['genres'] for genre in genres]))

{'Massively Multiplayer', 'Strategy', 'Free To Play', 'Racing', 'Adventure', 'Sports', 'Indie', 'RPG', 'Simulation', 'Action', 'Early Access', 'Casual'}


In [257]:
steamworks_df = steamworks_df[(steamworks_df['developers'].apply(lambda x: x != [])) &
                              (steamworks_df['publishers'].apply(lambda x: x != []))]

In [258]:
def strip_list_values(lst):
    return [item.strip() for item in lst if isinstance(item, str)]

steamworks_df['developers'] = steamworks_df['developers'].apply(strip_list_values)
steamworks_df['publishers'] = steamworks_df['publishers'].apply(strip_list_values)

Handling nulls

In [259]:
print(f"name (NULLS): {(steamworks_df['name'] == "Unknown").sum()}")
print(f"name (EMPTY): {(steamworks_df['name'] == "").sum()}")
print(f"developers (NULLS): {(steamworks_df['developers'].apply(lambda x: x == [])).sum()}")
print(f"publishers (NULLS): {steamworks_df['publishers'].apply(lambda x: x == []).sum()}")
print(f"initial_price (NULLS): {steamworks_df['initial_price'].isnull().sum()}")
print(f"final_price (NULLS): {steamworks_df['final_price'].isnull().sum()}")
print(f"platforms (NULLS): {steamworks_df['platforms'].apply(lambda x: x == {'linux': False, 'mac': False, 'windows': False}).sum()}")
print(f"metacritic (NULLS): {steamworks_df['metacritic'].isnull().sum()}")
print(f"genres (NULLS): {steamworks_df['genres'].apply(lambda x: x == []).sum()}")
print(f"release_date (NULLS): {(steamworks_df['release_date'] == "Unknown").sum()}")

name (NULLS): 0
name (EMPTY): 1
developers (NULLS): 0
publishers (NULLS): 0
initial_price (NULLS): 8882
final_price (NULLS): 8882
platforms (NULLS): 0
metacritic (NULLS): 15913
genres (NULLS): 25
release_date (NULLS): 0


In [260]:
app_id_to_drop = steamworks_df.loc[steamworks_df['name'] == '', 'appid']
print(app_id_to_drop.tolist())

[2942140]


In [261]:
steamworks_df = steamworks_df[~steamworks_df['appid'].isin(app_id_to_drop)]

In [262]:
null_genres = steamworks_df.loc[steamworks_df['genres'].apply(lambda x: len(x) == 0)]
print(null_genres)

        appid                           name             developers  \
1730   955900  Amazing Cultivation Simulator            [Gsq Games]   
2363  1155330                    Showgunners            [Artificer]   
2673  1247100          Spellmaster: The Saga  [Spellbook Creations]   
2705  1256690               Hot And Lovely 2         [Lovely Games]   
2863  1290070               Hot And Lovely 3         [Lovely Games]   
3071  1347430                     Cute Honey         [Lovely Games]   
3270  1399370                   Cute Honey 2         [Lovely Games]   
4154  1605010                 Adorable Witch         [Lovely Games]   
4548  1709460               Hot And Lovely 4         [Lovely Games]   
4549  1709500               Adorable Witch 2         [Lovely Games]   
4691  1740000         Cute Honey: Bunny Girl         [Lovely Games]   
5218  1864070         Yume : Special Edition         [Lovely Games]   
5310  1883530                    Snackhunter       [Polypirates Ug]   
5376  

In [263]:
steamworks_df = steamworks_df.drop(null_genres.index)

In [264]:
print(steamworks_df['developers'][0])
print(steamworks_df['publishers'][0])
print(steamworks_df['platforms'][0])
print(steamworks_df['name'][0])
print(steamworks_df['initial_price'][0])
print(steamworks_df['final_price'][0])
print(steamworks_df['metacritic'][0])
print(steamworks_df['release_date'][0])
print(steamworks_df['genres'][0])

['Valve']
['Valve']
{'linux': True, 'mac': True, 'windows': True}
Counter-Strike
999.0
999.0
88.0
Nov 1, 2000
[{'description': 'Action', 'id': '1'}]


Splitting dataframe

In [265]:
names = []
developers = []
publishers = []
game_prices = {}
genres = []
platforms = []
metacritic_scores = []
release_dates = []

for index, row in steamworks_df.iterrows():
    game_id = row['appid']
    
    names.append({'game_id': game_id, 'name': str(row['name'])})
    release_dates.append({'game_id': game_id, 'release_date': str(row['release_date'])})
    
    for developer in row['developers']:
        if developer:
            developers.append({'game_id': game_id, 'developer': str(developer)})
    
    for publisher in row['publishers']:
        if publisher:
            publishers.append({'game_id': game_id, 'publisher': str(publisher)})
    
    if row['platforms'].get('linux', False):
        platforms.append({'game_id': game_id, 'platform': 'linux'})
    if row['platforms'].get('mac', False):
        platforms.append({'game_id': game_id, 'platform': 'mac'})
    if row['platforms'].get('windows', False):
        platforms.append({'game_id': game_id, 'platform': 'windows'})
    
    for genre in row['genres']:
        genres.append({'game_id': game_id, 'genre': str(genre['description'])})

    game_prices[game_id] = {'initial_price': float(row['initial_price']), 'current_price': float(row['final_price'])}

    if not math.isnan(row['metacritic']):
        metacritic_scores.append({'game_id': game_id, 'score': float(row['metacritic'])})

Steam Spy API Data

In [266]:
game_ids = steamworks_df['appid']

In [267]:
data = pq.read_table('../data/steam_spy_details.parquet')
steamspy_df = pa.Table.to_pandas(data)
steamspy_df = steamspy_df[steamspy_df['appid'].isin(game_ids)]
steamspy_df.head()

,appid,score_rank,positive,negative,userscore,owners,average_playtime,average_playtime_2weeks,median_playtime,median_playtime_2weeks,ccu,current_price,initial_price
0,10,,236255,6239,0,"10,000,000 .. 20,000,000",0,0,0,0,14998,999,999
1,20,,7330,1093,0,"5,000,000 .. 10,000,000",0,0,0,0,62,499,499
2,30,,6267,672,0,"500,000 .. 1,000,000",0,0,0,0,98,499,499
3,40,,2553,527,0,"5,000,000 .. 10,000,000",0,0,0,0,4,499,499
4,50,,22337,1116,0,"2,000,000 .. 5,000,000",0,0,0,0,101,499,499


In [268]:
steamspy_df['appid'] = steamspy_df['appid'].astype(int)

In [269]:
steamspy_df = steamspy_df[~steamspy_df['appid'].isin(app_id_to_drop)]

datatypes

In [270]:
steamspy_df.dtypes

appid                       int64
score_rank                 object
positive                    int32
negative                    int32
userscore                   int32
owners                     object
average_playtime            int32
average_playtime_2weeks     int32
median_playtime             int32
median_playtime_2weeks      int32
ccu                         int32
current_price              object
initial_price              object
dtype: object

Handling nulls

In [271]:
print(f'Rank: {steamspy_df['score_rank'].apply(lambda x:x == '' or x== None).sum()}')
print(f'Positive: {steamspy_df['positive'].apply(lambda x: x == 0 or x == None or math.isnan(x)).sum()}')
print(f'Negative: {steamspy_df['negative'].apply(lambda x: x == 0 or x == None or math.isnan(x)).sum()}')
print(f'User Score: {steamspy_df['userscore'].apply(lambda x:x == 0 or x == None).sum()}')
print(f'Owners: {steamspy_df['owners'].apply(lambda x: x == '' or x==None).sum()}')
print(f'Avg Playtime: {steamspy_df['average_playtime'].apply(lambda x:x == 0 or x==None).sum()}')
print(f'Avg Playtime 2weeks: {steamspy_df['average_playtime_2weeks'].apply(lambda x:x == 0 or x==None).sum()}')
print(f'Median Playtime: {steamspy_df['median_playtime'].apply(lambda x:x == 0 or x==None).sum()}')
print(f'Median Playtime 2weeks: {steamspy_df['median_playtime_2weeks'].apply(lambda x:x == 0 or x==None).sum()}')
print(f'CCU: {steamspy_df['ccu'].apply(lambda x: x==None or math.isnan(x)).sum()}')
print(f'Current Price: {steamspy_df['current_price'].apply(lambda x: x==None or x=='').sum()}')
print(f'Initial Price: {steamspy_df['initial_price'].apply(lambda x: x==None or x=='').sum()}')

Rank: 16529
Positive: 9830
Negative: 10860
User Score: 16529
Owners: 0
Avg Playtime: 16534
Avg Playtime 2weeks: 16534
Median Playtime: 16534
Median Playtime 2weeks: 16534
CCU: 0
Current Price: 8506
Initial Price: 8506


Dropping empty columns

In [272]:
steamspy_df.drop('score_rank', axis=1, inplace=True)
steamspy_df.drop('userscore', axis=1, inplace=True)
steamspy_df.drop('average_playtime', axis=1, inplace=True)
steamspy_df.drop('average_playtime_2weeks', axis=1, inplace=True)
steamspy_df.drop('median_playtime', axis=1, inplace=True)
steamspy_df.drop('median_playtime_2weeks', axis=1, inplace=True)


In [273]:
steamspy_df.head()

,appid,positive,negative,owners,ccu,current_price,initial_price
0,10,236255,6239,"10,000,000 .. 20,000,000",14998,999,999
1,20,7330,1093,"5,000,000 .. 10,000,000",62,499,499
2,30,6267,672,"500,000 .. 1,000,000",98,499,499
3,40,2553,527,"5,000,000 .. 10,000,000",4,499,499
4,50,22337,1116,"2,000,000 .. 5,000,000",101,499,499


Compare and combine prices

In [274]:
steamworks_df.sort_values(by=['final_price'], ascending= False).head(25)


,appid,name,developers,publishers,initial_price,final_price,platforms,metacritic,genres,release_date
547,422610,1775: Rebellion,[Hexwar Games],[Hunted Cow Games],248000.0,248000.0,"{'linux': False, 'mac': False, 'windows': True}",NaN,"[{'description': 'Strategy', 'id': '2'}]","29 Sep, 2016"
610,449460,Commands & Colors: The Great War,[Hexwar Games],[Hunted Cow Games],198000.0,198000.0,"{'linux': False, 'mac': False, 'windows': True}",NaN,"[{'description': 'Strategy', 'id': '2'}]","19 Jan, 2017"
4282,1643320,S.T.A.L.K.E.R. 2: Heart Of Chornobyl,[Gsc Game World],"[Gsc Game World (Worldwide), Sega (Japan), 4Di...",139900.0,139900.0,"{'linux': False, 'mac': False, 'windows': True}",NaN,"[{'description': 'Action', 'id': '1'}, {'descr...",20 листоп. 2024
7328,2300320,Farming Simulator 25,[Giants Software],[Giants Software],99900.0,99900.0,"{'linux': False, 'mac': True, 'windows': True}",NaN,"[{'description': 'Simulation', 'id': '28'}]",12 листоп. 2024
186,244850,Space Engineers,[Keen Software House],[Keen Software House],83500.0,83500.0,"{'linux': False, 'mac': False, 'windows': True}",NaN,"[{'description': 'Action', 'id': '1'}, {'descr...","1 Mar, 2019"
4206,1620720,The Forest Quartet,[Mads & Friends],[Bedtime Digital Games],48000.0,48000.0,"{'linux': False, 'mac': False, 'windows': True}",NaN,"[{'description': 'Adventure', 'id': '25'}, {'d...","8 Dec, 2022"
11239,2829060,Computer Fury,[Saj Game Electronic],[Saj Game Electronic],40000.0,40000.0,"{'linux': False, 'mac': False, 'windows': True}",NaN,"[{'description': 'Action', 'id': '1'}, {'descr...","16 Mar, 2024"
99,105600,Terraria,[Re-Logic],[Re-Logic],38500.0,38500.0,"{'linux': True, 'mac': True, 'windows': True}",83.0,"[{'description': 'Action', 'id': '1'}, {'descr...","17 May, 2011"
101,107410,Arma 3,[Bohemia Interactive],[Bohemia Interactive],189000.0,37800.0,"{'linux': False, 'mac': True, 'windows': True}",74.0,"[{'description': 'Action', 'id': '1'}, {'descr...","12 Sep, 2013"
4588,1716310,Awaken - Astral Blade,[Dark Pigeon Games],[Esdigital Games],22799.0,22799.0,"{'linux': False, 'mac': False, 'windows': True}",NaN,"[{'description': 'Action', 'id': '1'}, {'descr...","22 Oct, 2024"


In [275]:
steamspy_df['initial_price'] = steamspy_df['initial_price'].fillna(0).astype(float)
steamspy_df['current_price'] = steamspy_df['current_price'].fillna(0).astype(float)
steamspy_df['initial_price'] = steamspy_df['initial_price'].astype(float)
steamspy_df['current_price'] = steamspy_df['current_price'].astype(float)
steamspy_df.sort_values(by=['initial_price'], ascending= False).head(25)

,appid,positive,negative,owners,ccu,current_price,initial_price
2860,1289890,4,4,"0 .. 20,000",0,19999.0,19999.0
12074,2914930,0,0,"0 .. 20,000",0,19999.0,19999.0
16126,3230870,0,0,"0 .. 20,000",0,11999.0,19999.0
16360,3244710,6,1,"0 .. 20,000",0,11999.0,19999.0
16359,3244700,5,2,"0 .. 20,000",0,11999.0,19999.0
5902,2014780,2872,604,"100,000 .. 200,000",511,7999.0,7999.0
7599,2340720,4,5,"0 .. 20,000",0,7999.0,7999.0
10054,2679460,14829,1485,"500,000 .. 1,000,000",11776,6999.0,6999.0
8988,2537590,0,0,"0 .. 20,000",0,6999.0,6999.0
8293,2440510,3521,4628,"1,000,000 .. 2,000,000",518,6999.0,6999.0


In [276]:
steam_spy_prices = steamspy_df[['appid', 'current_price', 'initial_price']]
steamworks_prices = steamworks_df[['appid', 'final_price', 'initial_price']]
steamworks_prices.head()

,appid,final_price,initial_price
0,10,999.0,999.0
1,20,499.0,499.0
2,30,499.0,499.0
3,40,499.0,499.0
4,50,499.0,499.0


In [277]:
game_prices = (
    steam_spy_prices
    .merge(steamworks_prices[['appid', 'final_price', 'initial_price']], on = 'appid', how = 'left', suffixes=('_spy', '_works'))
)
game_prices.head()

,appid,current_price,initial_price_spy,final_price,initial_price_works
0,10,999.0,999.0,999.0,999.0
1,20,499.0,499.0,499.0,499.0
2,30,499.0,499.0,499.0,499.0
3,40,499.0,499.0,499.0,499.0
4,50,499.0,499.0,499.0,499.0


In [278]:
game_prices['current_price'] = np.where(
    game_prices['current_price'] == 0,
    game_prices['final_price'],
    game_prices['current_price']
)

game_prices['initial_price'] = np.where(
    game_prices['initial_price_spy'] == 0,
    game_prices['initial_price_works'],
    game_prices['initial_price_spy']
)
game_prices = game_prices[['appid', 'current_price', 'initial_price']]
game_prices['current_price'] = game_prices['current_price'] / 100
game_prices['initial_price'] = game_prices['initial_price'] / 100
game_prices.rename(columns={'appid': 'game_id'}, inplace= True)
game_prices.sort_values(by = ['initial_price'], ascending = False).head(25)

,game_id,current_price,initial_price
14974,3244710,119.99,199.99
11144,2914930,199.99,199.99
14973,3244700,119.99,199.99
14775,3230890,199.99,199.99
14534,3214810,199.99,199.99
14533,3214800,199.99,199.99
2677,1289890,199.99,199.99
14773,3230870,119.99,199.99
7074,2340720,79.99,79.99
5508,2014780,79.99,79.99


In [279]:
del steam_spy_prices
del steamworks_prices

Splitting dataframe

In [280]:
user_reviews = []
owners = []
ccu = []

for index, row in steamspy_df.iterrows():
    game_id = int(row['appid'])
    
    user_reviews.append({
        'game_id': game_id,
        'positive': int(row['positive']) if not math.isnan(row['positive']) else float('nan'),
        'negative': int(row['negative']) if not math.isnan(row['negative']) else float('nan')
    })
    
    owners.append({'game_id': game_id, 'owners': str(row['owners'])})
    ccu.append({'game_id': game_id, 'ccu': int(row['ccu'])})


In [281]:
games_df = pd.DataFrame(names)
dates_df = pd.DataFrame(release_dates)
developers_df = pd.DataFrame(developers)
publishers_df = pd.DataFrame(publishers)
platforms_df = pd.DataFrame(platforms)
genres_df = pd.DataFrame(genres)
metacritic_scores_df = pd.DataFrame(metacritic_scores)
user_reviews_df = pd.DataFrame(user_reviews)
owners_df = pd.DataFrame(owners)
ccu_df = pd.DataFrame(ccu)
game_prices_df = pd.DataFrame(game_prices)

In [282]:
developers_df['developer'] = developers_df['developer'].str.strip()
publishers_df['publisher'] = publishers_df['publisher'].str.strip()
null_developers = developers_df['developer'].isna().sum()
null_publishers = publishers_df['publisher'].isna().sum()
empty_developers = (developers_df['developer'] == '').sum()
empty_publishers = (publishers_df['publisher'] == '').sum()
print("# of Rows with NULL publishers:", null_publishers)
print("# of Rows with NULL developers:", null_developers)
print(f"# of Empty values in 'publisher': {empty_publishers}")
print(f"# of Empty values in 'developer': {empty_developers}")
print(f"developer (EMPTY): {(developers_df['developer'] == "").sum()}")
print(f"publisher (EMPTY): {(publishers_df['publisher'] == "").sum()}")
print(publishers_df['publisher'].apply(type).value_counts())


# of Rows with NULL publishers: 0
# of Rows with NULL developers: 0
# of Empty values in 'publisher': 0
# of Empty values in 'developer': 0
developer (EMPTY): 0
publisher (EMPTY): 0
publisher
<class 'str'>    17487
Name: count, dtype: int64


In [283]:
non_printable_pattern = re.compile(r'[^\x20-\x7E]')
unique_pub = publishers_df['publisher'].unique().tolist()
problematic_publishers = [pub for pub in unique_pub if non_printable_pattern.search(pub)]
print(problematic_publishers)

[]


In [284]:
print(f'{games_df.head()}\n')
print(f'{game_prices_df.head()}\n')
print(f'{platforms_df.head()}\n')
print(f'{metacritic_scores_df.head()}\n')
print(f'{dates_df.head()}\n')
print(f'{user_reviews_df.head()}\n')
print(f'{owners_df.head()}\n')
print(f'{ccu_df.head()}\n')
print(f'{developers_df.head()}\n')
print(f'{publishers_df.head()}\n')
print(f'{genres_df.head()}\n')


   game_id                       name
0       10             Counter-Strike
1       20      Team Fortress Classic
2       30              Day Of Defeat
3       40         Deathmatch Classic
4       50  Half-Life: Opposing Force

   game_id  current_price  initial_price
0       10           9.99           9.99
1       20           4.99           4.99
2       30           4.99           4.99
3       40           4.99           4.99
4       50           4.99           4.99

   game_id platform
0       10    linux
1       10      mac
2       10  windows
3       20    linux
4       20      mac

   game_id  score
0       10   88.0
1       30   79.0
2       70   96.0
3       80   65.0
4      240   88.0

   game_id release_date
0       10  Nov 1, 2000
1       20  Apr 1, 1999
2       30  May 1, 2003
3       40  Jun 1, 2001
4       50  Nov 1, 1999

   game_id  positive  negative
0       10    236255      6239
1       20      7330      1093
2       30      6267       672
3       40      2553     

Splitting owners into min and max values

In [285]:
owners_df['owners'][0].split(" .. ")[1].strip()

'20,000,000'

Merging dataframes

In [286]:
game_metrics_df = (
    games_df
    .merge(user_reviews_df[['game_id', 'positive', 'negative']], on = 'game_id', how = 'left')
    .merge(metacritic_scores_df[['game_id', 'score']], on = 'game_id', how = 'left')
    .merge(ccu_df[['game_id', 'ccu']], on = 'game_id', how = 'left')
    .merge(owners_df[['game_id', 'owners']], on = 'game_id', how= 'left')
)

game_metrics_df[['owners_min', 'owners_max']] = (
    game_metrics_df['owners']
    .str.split(" .. ", expand = True)
    .apply(lambda x: x.str.replace(',','').astype(int))
)

game_metrics_df = game_metrics_df.drop(columns=['owners'])
game_metrics_df = game_metrics_df.rename(columns={'score': 'metacritic_score', 'positive': 'positive_user_score', 'negative': 'negative_user_score'})

Normalizing release dates

In [287]:
correct_pattern = r'^[A-Z][a-z]{2} \d{1,2}, \d{4}$'
non_matching_dates = dates_df[~dates_df['release_date'].str.match(correct_pattern, na=False)]
non_matching_dates_count = non_matching_dates.shape[0]
print(f"Number of dates not matching: {non_matching_dates_count}")
pd.set_option('display.max_rows', None)
print(non_matching_dates)


Number of dates not matching: 7030
       game_id     release_date
65       40960                 
86      105600     17 May, 2011
87      107100     16 Aug, 2011
88      107410     12 Sep, 2013
139     230410     26 Mar, 2013
154     237930     20 May, 2014
164     244850      1 Mar, 2019
169     247710  To be announced
173     252190      Coming soon
211     275390     21 Aug, 2014
227     282800     16 May, 2014
236     292990         Aug 2014
247     301280  To be announced
255     304430      7 Jul, 2016
256     304770             2025
257     305620      1 Aug, 2017
261     306910  To be announced
317     340020  To be announced
337     348930             2024
404     374780      Coming soon
443     395830      Coming soon
449     397950     27 Sep, 2016
477     413410     18 Feb, 2016
488     420530         Dec 2016
495     422610     29 Sep, 2016
513     431170      Coming soon
520     433220          Q4 2024
553     448840             2025
554     449460     19 Jan, 2017
571  

In [288]:
english_months = {
    'листоп.': 'Nov'
}
for non_english, english in english_months.items():
    dates_df['release_date'] = dates_df['release_date'].str.replace(non_english, english, regex=False)

def reformat_dates(date):
    try:
        if pd.Series(date).str.match(r'^[A-Z][a-z]{2} \d{1,2}, \d{4}$').iloc[0]:
            dt = pd.to_datetime(date, format='%b %d, %Y')
        elif pd.Series(date).str.match(r'^\d{1,2} [A-Z][a-z]{2}, \d{4}$').iloc[0]:
            dt = pd.to_datetime(date, format='%d %b, %Y')
        else:
            dt = pd.to_datetime("Jan 1, 2025")
        
        if dt > pd.Timestamp("2024-11-12"):
            dt = pd.to_datetime("Jan 1, 2025")

        return dt
    except Exception:
        print("weird date:", date)
        return pd.to_datetime("Jan 1, 2025")

dates_df['release_date'] = dates_df['release_date'].apply(reformat_dates)


weird date: Apr 3, 4444


Handling NULL prices based on un released games

In [289]:
price_dates_df = (
    dates_df
    .merge(game_prices_df, on = 'game_id', how = 'left')
)

target_date = pd.to_datetime("Jan 1, 2025")
price_dates_df['current_price'] = np.where(
    (price_dates_df['release_date'] != target_date) & (price_dates_df['current_price'].isna()),
    0,
    np.where(price_dates_df['current_price'].isna(), -1, price_dates_df['current_price'])
)

target_date = pd.to_datetime("Jan 1, 2025")
price_dates_df['initial_price'] = np.where(
    (price_dates_df['release_date'] != target_date) & (price_dates_df['initial_price'].isna()),
    0,
    np.where(price_dates_df['initial_price'].isna(), -1, price_dates_df['initial_price'])
)

Merging Dates & prices

In [290]:
game_metrics_df = (
    game_metrics_df
    .merge(price_dates_df, on = 'game_id', how = 'left')
)
game_metrics_df.head()

,game_id,name,positive_user_score,negative_user_score,metacritic_score,ccu,owners_min,owners_max,release_date,current_price,initial_price
0,10,Counter-Strike,236255,6239,88.0,14998,10000000,20000000,2000-11-01,9.99,9.99
1,20,Team Fortress Classic,7330,1093,NaN,62,5000000,10000000,1999-04-01,4.99,4.99
2,30,Day Of Defeat,6267,672,79.0,98,500000,1000000,2003-05-01,4.99,4.99
3,40,Deathmatch Classic,2553,527,NaN,4,5000000,10000000,2001-06-01,4.99,4.99
4,50,Half-Life: Opposing Force,22337,1116,NaN,101,2000000,5000000,1999-11-01,4.99,4.99


Handling duplicates

In [291]:
developers_df['developer'] = developers_df['developer'].str.lower()
publishers_df['publisher'] = publishers_df['publisher'].str.lower()

In [292]:
duplicated_metrics = game_metrics_df[game_metrics_df.duplicated(keep=False)]
print(f"duplicated metrics: {duplicated_metrics}\n")
duplicated_game_ids_df = game_metrics_df[game_metrics_df.duplicated(subset=['game_id'], keep=False)]
print(f"duplicated game ids (metrics): {duplicated_game_ids_df}\n")

duplicated_games = games_df[games_df.duplicated(keep=False)]
print(f"duplicated games: {duplicated_games}\n")
duplicated_game_ids_df = games_df[games_df.duplicated(subset=['game_id'], keep=False)]
print(f"duplicated game ids (games): {duplicated_game_ids_df}\n")

duplicated_platforms = platforms_df[platforms_df.duplicated(keep=False)]
print(f"duplicated platforms: {duplicated_platforms}\n")

duplicated_developers = developers_df[developers_df.duplicated(keep=False)]
print(f"duplicated developers: {duplicated_developers}\n")

duplicated_publishers = publishers_df[publishers_df.duplicated(keep=False)]
print(f"duplicated publishers: {duplicated_publishers}\n")

duplicated_genres = genres_df[genres_df.duplicated(keep=False)]
print(f"duplicated genres: {duplicated_genres}\n")


duplicated metrics: Empty DataFrame
Columns: [game_id, name, positive_user_score, negative_user_score, metacritic_score, ccu, owners_min, owners_max, release_date, current_price, initial_price]
Index: []

duplicated game ids (metrics): Empty DataFrame
Columns: [game_id, name, positive_user_score, negative_user_score, metacritic_score, ccu, owners_min, owners_max, release_date, current_price, initial_price]
Index: []

duplicated games: Empty DataFrame
Columns: [game_id, name]
Index: []

duplicated game ids (games): Empty DataFrame
Columns: [game_id, name]
Index: []

duplicated platforms: Empty DataFrame
Columns: [game_id, platform]
Index: []

duplicated developers:        game_id            developer
7752   2342690            tomatoast
7753   2342690            tomatoast
9040   2532590           bronstudio
9041   2532590           bronstudio
10805  2775370      disillusion dev
10806  2775370      disillusion dev
11901  2896220         j.a.p studio
11902  2896220         j.a.p studio
136

In [293]:
rows_with_appid = steamworks_df[steamworks_df['appid'] == 2342690]
print(rows_with_appid)
rows_with_appid = steamworks_df[steamworks_df['appid'] == 3148790]
print(rows_with_appid)

        appid          name              developers   publishers  \
7611  2342690  Call Of Boba  [Tomatoast, Tomatoast]  [Tomatoast]   

      initial_price  final_price  \
7611            NaN          NaN   

                                            platforms  metacritic  \
7611  {'linux': False, 'mac': False, 'windows': True}         NaN   

                                                 genres release_date  
7611  [{'description': 'Action', 'id': '1'}, {'descr...  Coming soon  
         appid       name developers                            publishers  \
14791  3148790  Back Door  [Akihire]  [The Lone Warrior, The Lone Warrior]   

       initial_price  final_price  \
14791            NaN          NaN   

                                             platforms  metacritic  \
14791  {'linux': False, 'mac': False, 'windows': True}         NaN   

                                                  genres  release_date  
14791  [{'description': 'Adventure', 'id': '25'}, {'d...  Mar 3

In [294]:
developers_df = developers_df.drop_duplicates(subset=['game_id', 'developer'])
publishers_df = publishers_df.drop_duplicates(subset=['game_id', 'publisher'])

Creating Look up and bridge tables

In [295]:
developers_lookup_df = developers_df[['developer']].drop_duplicates().reset_index(drop=True)
developers_lookup_df['developer_id'] = developers_lookup_df.index+1
game_developers_bridge_df = developers_df.merge(developers_lookup_df, on='developer', how='inner')
game_developers_bridge_df = game_developers_bridge_df[['game_id', 'developer_id']]

publishers_lookup_df = publishers_df[['publisher']].drop_duplicates().reset_index(drop=True)
publishers_lookup_df['publisher_id'] = publishers_lookup_df.index+1
game_publishers_bridge_df = publishers_df.merge(publishers_lookup_df, on='publisher', how='inner')
game_publishers_bridge_df = game_publishers_bridge_df[['game_id', 'publisher_id']]

genres_lookup_df = genres_df[['genre']].drop_duplicates().reset_index(drop=True)
genres_lookup_df['genre_id'] = genres_lookup_df.index+1
game_genres_bridge_df = genres_df.merge(genres_lookup_df, on='genre', how='inner')
game_genres_bridge_df = game_genres_bridge_df[['game_id', 'genre_id']]

platforms_lookup_df = platforms_df[['platform']].drop_duplicates().reset_index(drop=True)
platforms_lookup_df['platform_id'] = platforms_lookup_df.index+1
game_platforms_bridge_df = platforms_df.merge(platforms_lookup_df, on='platform', how='inner')
game_platforms_bridge_df = game_platforms_bridge_df[['game_id', 'platform_id']]

In [296]:
print(developers_lookup_df.head())
print(game_developers_bridge_df.head())
print(publishers_lookup_df.head())
print(game_publishers_bridge_df.head())
print(genres_lookup_df.head())
print(game_genres_bridge_df.head())
print(platforms_lookup_df .head())
print(game_platforms_bridge_df.head())


               developer  developer_id
0                  valve             1
1       gearbox software             2
2  malfador machinations             3
3               techland             4
4     popcap games, inc.             5
   game_id  developer_id
0       10             1
1       20             1
2       30             1
3       40             1
4       50             2
             publisher  publisher_id
0                valve             1
1       strategy first             2
2             techland             3
3  techland publishing             4
4   popcap games, inc.             5
   game_id  publisher_id
0       10             1
1       20             1
2       30             1
3       40             1
4       50             1
          genre  genre_id
0        Action         1
1  Free To Play         2
2      Strategy         3
3        Racing         4
4        Casual         5
   game_id  genre_id
0       10         1
1       20         1
2       30         1
3   

Write to CSV

In [297]:
os.makedirs('../data', exist_ok=True)

game_metrics_df.to_csv('../data/game_metrics.csv', sep =',', encoding = 'utf-8', index = False, header = True)

developers_lookup_df.to_csv('../data/developers_lookup.csv', sep=',', encoding='utf-8', index=False, header=True)
game_developers_bridge_df.to_csv('../data/game_developers_bridge.csv', sep=',', encoding='utf-8', index=False, header=True)

publishers_lookup_df.to_csv('../data/publishers_lookup.csv', sep=',', encoding='utf-8', index=False, header=True)
game_publishers_bridge_df.to_csv('../data/game_publishers_bridge.csv', sep=',', encoding='utf-8', index=False, header=True)

genres_lookup_df.to_csv('../data/genres_lookup.csv', sep=',', encoding='utf-8', index=False, header=True)
game_genres_bridge_df.to_csv('../data/game_genres_bridge.csv', sep=',', encoding='utf-8', index=False, header=True)

platforms_lookup_df.to_csv('../data/platforms_lookup.csv', sep =',', encoding = 'utf-8', index = False, header = True)
game_platforms_bridge_df.to_csv('../data/game_platforms_bridge.csv', sep=',', encoding='utf-8', index=False, header=True)